## Final Assignment

In [5]:
import pandas as pd 

df = pd.read_excel('SerialKiller_Dataset_with_Targeted_Group_Type(FINAL).xlsx')
df

,Name,Birthday,Countries Active,Years Active,Proven Victims,Possible Victims,BirthYear,StartYear,Age At The Start of Killing,Notes,Type,Nickname,Reason,Archetype/Classification,Gender,Victim Age,Victim Gender,Targeted Group Type
0,Harold Shipman,1/14/1946,United Kingdom,1975-1998,218,250,1946.0,1975.0,29.0,Convicted of 15 murders and responsible for th...,"Hate Crime, Medical",Dr Death Angel of Death,Power,Power/Control,Male,Adult,Mixed,NaN
1,Miyuki Ishikawa,NaN,Japan,1940s,103+,169,NaN,1940.0,NaN,Killed more than 103 newborn children. As a ma...,Medical,NaN,Power,Power/Control,Female,Child,Mixed,NaN
2,Niels HÃƒgel,12/30/1976,Germany,1999-2005,100,300,1976.0,1999.0,23.0,Nurse who was sentenced to life imprisonment f...,"Hate Crime, Medical",NaN,Power,Power/Control,Male,Adult,Male,NaN
3,Louay Omar Mohammed al-Taei,NaN,Iraq,2005-2006,43,NaN,NaN,2005.0,NaN,Medical doctor found to have killed 43 wounded...,Medical,NaN,Power,Power/Control,Male,Adult,Male,NaN
4,Donald Harvey,4/15/1952,United States,1970-1987,37,57-87,1952.0,1970.0,18.0,Self-professed Angel of Death. Worked as an or...,"Hate Crime, Medical",Angel of Death,Power,Power/Control,Male,Adult,Mixed,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,Martin Dumollard,4/21/1810,United Kingdom,1855-1861,3,3,1810.0,1855.0,45.0,Dumollard was arrested and charged with the de...,"Rapist, Strangler, Sexual Predator",The Maid Killer,Sexual Assault,Hedonistic - Lust,Male,Adult,Female,NaN
635,Paul Denyer,4/14/1972,Austrailia,1993,3,5,1972.0,1993.0,21.0,"In 1993, Paul Denyer was sentenced to three co...","Sexual Predator, Strangler","Paula Denyer, Frankston Killer",Sexual Assault,Hedonistic - Lust,Male,Adult,Female,NaN
636,Gerard John Schaefer,3/25/1946,United States,1969-1973,2,30,1946.0,1969.0,23.0,Boasted he impressed fellow inmate Ted Bundy i...,Unknown,Killer Fiction,Power,Power/Control,Male,Adult,Mixed,NaN
637,Lucious Boyd,3/22/1959,United States,1993-1999,1,10,1959.0,1993.0,34.0,"He was known to kidnap, rape stab and murder h...","Rapist, Sexual Predator","Lucilous Boyd, Lucifer",Sexual Assault,Hedonistic - Lust,Male,Adult,Male,NaN
